# Model Building

In this notebook, I will develop a predictive model to forecast future energy consumption. 

I will experiment with multiple regression algorithms, compare their performances using standard evaluation metrics (RMSE, MAE, R² Score), and select the best-performing model for deployment. This step is critical to ensure accurate and reliable energy demand forecasting.

### Key Steps:
- Load processed dataset
- Perform time-based train-test split
- Train multiple regression models
- Evaluate using RMSE, MAE, and R²
- Select and save the best model

In [1]:
## importing required libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

import warnings
warnings.filterwarnings('ignore')

In [2]:
## loading the dataset
df_cleaned = pd.read_parquet(r"C:\Users\himan\Desktop\Projects\Energy_Forecasting_System\data\processed-data\est_hourly_cleaned_with_features.parquet")
df_cleaned.head()

,AEP,COMED,DAYTON,DEOK,DOM,DUQ,EKPC,FE,NI,PJME,...,PJMW_rolling_std_24,PJMW_rolling_mean_168,PJMW_rolling_std_168,PJM_Load_lag_1,PJM_Load_lag_24,PJM_Load_rolling_mean_24,PJM_Load_rolling_std_24,PJM_Load_rolling_mean_168,PJM_Load_rolling_std_168,is_holiday
Datetime,,,,,,,,,,,,,,,,,,,,,
1998-04-11 12:00:00+00:00,12379.0,9631.0,1621.0,2533.0,7190.0,1364.0,1166.0,0.0,9198.0,30393.0,...,0.0,4374.0,0.0,25574.0,24824.0,24030.458333,2512.084064,25457.672619,3764.174539,0
1998-04-11 13:00:00+00:00,12379.0,9631.0,1621.0,2533.0,7190.0,1364.0,1166.0,0.0,9198.0,30393.0,...,0.0,4374.0,0.0,25179.0,22712.0,24106.916667,2498.096735,25477.333333,3750.635883,0
1998-04-11 14:00:00+00:00,12379.0,9631.0,1621.0,2533.0,7190.0,1364.0,1166.0,0.0,9198.0,30393.0,...,0.0,4374.0,0.0,24547.0,21245.0,24214.208333,2424.034420,25497.565476,3732.319990,0
1998-04-11 15:00:00+00:00,12379.0,9631.0,1621.0,2533.0,7190.0,1364.0,1166.0,0.0,9198.0,30393.0,...,0.0,4374.0,0.0,23820.0,21150.0,24299.458333,2346.316632,25512.345238,3718.149311,0
1998-04-11 16:00:00+00:00,12379.0,9631.0,1621.0,2533.0,7190.0,1364.0,1166.0,0.0,9198.0,30393.0,...,0.0,4374.0,0.0,23196.0,21188.0,24363.333333,2277.795247,25513.505952,3717.241792,0


Now its time to split the data into "train" for training the model and "test" for testing it on the model. But unlike other cases where we split the data randomly, in time-series tasks, we need to take care that we only want to train our model on past data and test on the future data. This can prevent data leakage.  

In [3]:
df_cleaned.columns

Index(['AEP', 'COMED', 'DAYTON', 'DEOK', 'DOM', 'DUQ', 'EKPC', 'FE', 'NI',
       'PJME', 'PJMW', 'PJM_Load', 'hour', 'day_of_week', 'month',
       'day_of_year', 'is_weekend', 'AEP_lag_1', 'AEP_lag_24',
       'AEP_rolling_mean_24', 'AEP_rolling_std_24', 'AEP_rolling_mean_168',
       'AEP_rolling_std_168', 'COMED_lag_1', 'COMED_lag_24',
       'COMED_rolling_mean_24', 'COMED_rolling_std_24',
       'COMED_rolling_mean_168', 'COMED_rolling_std_168', 'DAYTON_lag_1',
       'DAYTON_lag_24', 'DAYTON_rolling_mean_24', 'DAYTON_rolling_std_24',
       'DAYTON_rolling_mean_168', 'DAYTON_rolling_std_168', 'DEOK_lag_1',
       'DEOK_lag_24', 'DEOK_rolling_mean_24', 'DEOK_rolling_std_24',
       'DEOK_rolling_mean_168', 'DEOK_rolling_std_168', 'DOM_lag_1',
       'DOM_lag_24', 'DOM_rolling_mean_24', 'DOM_rolling_std_24',
       'DOM_rolling_mean_168', 'DOM_rolling_std_168', 'DUQ_lag_1',
       'DUQ_lag_24', 'DUQ_rolling_mean_24', 'DUQ_rolling_std_24',
       'DUQ_rolling_mean_168', 'DUQ_rollin

In [7]:
df_cleaned.drop(columns=['EKPC_lag_1',
       'EKPC_lag_24', 'EKPC_rolling_mean_24', 'EKPC_rolling_std_24',
       'EKPC_rolling_mean_168', 'EKPC_rolling_std_168', 'EKPC'])


,AEP,COMED,DAYTON,DEOK,DOM,DUQ,FE,NI,PJME,PJMW,...,PJMW_rolling_std_24,PJMW_rolling_mean_168,PJMW_rolling_std_168,PJM_Load_lag_1,PJM_Load_lag_24,PJM_Load_rolling_mean_24,PJM_Load_rolling_std_24,PJM_Load_rolling_mean_168,PJM_Load_rolling_std_168,is_holiday
Datetime,,,,,,,,,,,,,,,,,,,,,
1998-04-11 12:00:00+00:00,12379.0,9631.0,1621.0,2533.0,7190.0,1364.0,0.0,9198.0,30393.0,4374.0,...,0.000000,4374.000000,0.000000,25574.0,24824.0,24030.458333,2512.084064,25457.672619,3764.174539,0
1998-04-11 13:00:00+00:00,12379.0,9631.0,1621.0,2533.0,7190.0,1364.0,0.0,9198.0,30393.0,4374.0,...,0.000000,4374.000000,0.000000,25179.0,22712.0,24106.916667,2498.096735,25477.333333,3750.635883,0
1998-04-11 14:00:00+00:00,12379.0,9631.0,1621.0,2533.0,7190.0,1364.0,0.0,9198.0,30393.0,4374.0,...,0.000000,4374.000000,0.000000,24547.0,21245.0,24214.208333,2424.034420,25497.565476,3732.319990,0
1998-04-11 15:00:00+00:00,12379.0,9631.0,1621.0,2533.0,7190.0,1364.0,0.0,9198.0,30393.0,4374.0,...,0.000000,4374.000000,0.000000,23820.0,21150.0,24299.458333,2346.316632,25512.345238,3718.149311,0
1998-04-11 16:00:00+00:00,12379.0,9631.0,1621.0,2533.0,7190.0,1364.0,0.0,9198.0,30393.0,4374.0,...,0.000000,4374.000000,0.000000,23196.0,21188.0,24363.333333,2277.795247,25513.505952,3717.241792,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-08-02 20:00:00+00:00,17673.0,12151.0,2554.0,3120.5,14038.0,1966.0,9866.0,10042.0,44057.0,6545.0,...,762.020384,5677.011905,855.481585,35082.0,35082.0,35082.000000,0.000000,35082.000000,0.000000,0
2018-08-02 21:00:00+00:00,17303.0,12151.0,2481.0,3120.5,13832.0,1944.0,9656.0,10042.0,43256.0,6496.0,...,758.908101,5673.636905,851.088084,35082.0,35082.0,35082.000000,0.000000,35082.000000,0.000000,0
2018-08-02 22:00:00+00:00,17001.0,12151.0,2405.0,3120.5,13312.0,1901.0,9532.0,10042.0,41552.0,6325.0,...,757.766507,5670.244048,847.317772,35082.0,35082.0,35082.000000,0.000000,35082.000000,0.000000,0


In [8]:
df_cleaned['AEP_lag_1'] = df_cleaned['AEP'].shift(1)
df_cleaned['AEP_rolling_mean_24'] = df_cleaned['AEP'].rolling(window=24).mean().shift(1)  # shift(1) is critical
df_cleaned = df_cleaned.dropna()

In [9]:
## splitting the data into train and test
test_size = 0.8
split_range = int(test_size * len(df_cleaned))  ## 0.8 * x can give us float value which will give us an error in train-test split. So we make sure that it returns only int.

In [10]:
import numpy as np
arr = df_cleaned.astype(np.float32)

In [11]:
## defining target variable
X = df_cleaned.drop(["PJM_Load"] , axis = 1)
y = df_cleaned["PJM_Load"]

In [12]:
X[['AEP_lag_1', 'AEP_rolling_mean_24']].head()

,AEP_lag_1,AEP_rolling_mean_24
Datetime,,
1998-04-13 00:00:00+00:00,12379.0,12379.0
1998-04-13 01:00:00+00:00,12379.0,12379.0
1998-04-13 02:00:00+00:00,12379.0,12379.0
1998-04-13 03:00:00+00:00,12379.0,12379.0
1998-04-13 04:00:00+00:00,12379.0,12379.0


In [13]:
## Time-based Train-Test split
X_train, X_test = X.iloc[:split_range], X.iloc[split_range:]
y_train, y_test = y.iloc[:split_range], y.iloc[split_range:]

In [14]:
## importing metrices
from sklearn.metrics import mean_absolute_error, mean_squared_error, root_mean_squared_error, r2_score

In [15]:
## creating function to evalueate model
import numpy as np
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, mse, rmse, r2_square

In [16]:
## our models
models = {
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor(),
}

In [18]:
## building the model
result = []

for model_name, model in models.items():
    model.fit(X_train, y_train)

    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    train_metrices = evaluate_model(y_train, y_train_pred)
    test_metrices = evaluate_model(y_test, y_test_pred)

    result.append({
        'Model': model_name,
        'Train_RMSE': train_metrices[2],
        'Test_RMSE': test_metrices[2],
        'Train_R2': train_metrices[3],
        'Test_R2': test_metrices[3]
    })

### Training on SARIMA - Seasonal ARIMA 

In [17]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
df_sarima = df_cleaned[['PJM_Load']]
# Fit the SARIMA model (adjust the parameters based on your data, p,d,q, seasonal_order)
sarima_model = SARIMAX(y_train,
                       order=(1, 1, 1),  # p, d, q values (adjust as necessary)
                       seasonal_order=(1, 1, 1, 24),  # P, D, Q, S for daily seasonality
                       enforce_stationarity=False,
                       enforce_invertibility=False)

sarima_fitted = sarima_model.fit(disp=False)

predictions_sarima = sarima_fitted.predict(start=len(y_train), end=len(y_train) + len(y_test) - 1, dynamic=False)

# Evaluate the performance of the model
train_metrices_sarima = evaluate_model(y_train, y_train_pred)
test_metrices_sarima = evaluate_model(y_test, y_test_pred)

result.append({
    'Model': model_name,
    'Train_RMSE': train_metrices[2],
    'Test_RMSE': test_metrices[2],
    'Train_R2': train_metrices[3],
    'Test_R2': test_metrices[3]})

C:\Users\himan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\himan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\himan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\tsa\base\tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.


MemoryError: Unable to allocate 643. MiB for an array with shape (51, 51, 32378) and data type float64

### Training on SARIMAX - Seasonal ARIMA 

In [ ]:
X = df_cleaned[['AEP', 'COMED', 'DAYTON']]
df_sarima = df_cleaned[['PJM_Load']]

# Fit the SARIMAX model with exogenous variables
sarimax_model = SARIMAX(y_train,
                        exog=X_train,
                        order=(1, 1, 1),  # p, d, q values (adjust as necessary)
                        seasonal_order=(1, 1, 1, 24),  # P, D, Q, S for daily seasonality
                        enforce_stationarity=False,
                        enforce_invertibility=False)

sarimax_fitted = sarimax_model.fit(disp=False)

predictions_sarimax = sarimax_fitted.predict(start=len(y_train), end=len(y_train) + len(y_test) - 1, exog=X_test, dynamic=False)

# Evaluate the performance of the model
train_metrices_sarimax = evaluate_model(y_train, y_train_pred)
test_metrices_sarimax = evaluate_model(y_test, y_test_pred)

result.append({
    'Model': model_name,
    'Train_RMSE': train_metrices[2],
    'Test_RMSE': test_metrices[2],
    'Train_R2': train_metrices[3],
    'Test_R2': test_metrices[3]})

C:\Users\himan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\himan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


### Prophet

In [ ]:
from fbprophet import Prophet


prophet_data = df_cleaned[['PJM_Load']].reset_index()
prophet_data.columns = ['ds', 'y']  # 'ds' is the datetime column, 'y' is the target variable

# Initialize the Prophet model
prophet_model = Prophet(daily_seasonality=True, yearly_seasonality=True, seasonality_mode='multiplicative')

# Fit the model
prophet_model.fit(prophet_data)

# Make predictions (for next 24 hours as an example)
future = prophet_model.make_future_dataframe(prophet_data, periods=24, freq='H')  # 24 hours ahead
forecast = prophet_model.predict(future)

# Evaluate the performance
prophet_predictions = forecast['yhat'][-len(y_test):].values  # Get the predictions for test period

# Evaluate the performance of the model
train_metrices_sarimax = evaluate_model(y_train, y_train_pred)
test_metrices_sarimax = evaluate_model(y_test, y_test_pred)

In [19]:
import pandas as pd

results_df = pd.DataFrame(result)
results_df = results_df.sort_values(by='Test_RMSE')
print(results_df)

                     Model  Train_RMSE  Test_RMSE  Train_R2  Test_R2
0            Decision Tree    0.000000        0.0  1.000000      1.0
1  Random Forest Regressor  146.288107        0.0  0.997468      1.0
